In [3]:
from rl import run_inference, ThorNavEnv, CLIPCuriosity, ExtrinsicReward
from models import ActorCritic
from cons import DEVICE
import torch
import prior
from ai2thor.controller import Controller

dataset = prior.load_dataset("procthor-10k")
train_scenes = dataset["train"]

NUM_UPDATES = 100

idx = torch.randint(0, len(train_scenes), (1,)).item()
idx = 3
house = train_scenes[idx]

controller = Controller(
    scene=house,
    snapToGrid=False,
    rotateStepDegrees=30,
    renderDepthImage=True,
)



Fetching reference HEAD


[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 20398.13it/s]


In [10]:

# Load trained model
actor_critic = ActorCritic().to(DEVICE)
actor_critic.load_state_dict(torch.load("data/ac_update_40.pt", map_location="cpu"))

clip_curiosity = CLIPCuriosity()
# Disable curiosity during inference
env = ThorNavEnv(controller, clip_curiosity=clip_curiosity, extrinsic_reward=ExtrinsicReward())

# Run inference
traj, total = run_inference(actor_critic, env, max_steps=256, deterministic=False)

print("Episode return:", total)

Episode return: 21.50148253356829


In [8]:
for i in traj:
    print(i["logits"].softmax(dim=1))

tensor([[0.4883, 0.2749, 0.2368]])
tensor([[0.5564, 0.2451, 0.1986]])
tensor([[0.5923, 0.2283, 0.1794]])
tensor([[0.6278, 0.2104, 0.1618]])
tensor([[0.6560, 0.1953, 0.1487]])
tensor([[0.6686, 0.1917, 0.1397]])
tensor([[0.6724, 0.1942, 0.1334]])
tensor([[0.6783, 0.1937, 0.1280]])
tensor([[0.6797, 0.1966, 0.1236]])
tensor([[0.6821, 0.1975, 0.1204]])
tensor([[0.6812, 0.2014, 0.1173]])
tensor([[0.6811, 0.2052, 0.1137]])
tensor([[0.6769, 0.2111, 0.1120]])
tensor([[0.6711, 0.2184, 0.1104]])
tensor([[0.6669, 0.2234, 0.1097]])
tensor([[0.6627, 0.2292, 0.1081]])
tensor([[0.6548, 0.2363, 0.1089]])
tensor([[0.6536, 0.2378, 0.1085]])
tensor([[0.6524, 0.2390, 0.1087]])
tensor([[0.6465, 0.2435, 0.1101]])
tensor([[0.6428, 0.2460, 0.1113]])
tensor([[0.6457, 0.2422, 0.1121]])
tensor([[0.6544, 0.2336, 0.1120]])
tensor([[0.6515, 0.2362, 0.1123]])
tensor([[0.6506, 0.2369, 0.1125]])
tensor([[0.6561, 0.2335, 0.1104]])
tensor([[0.6556, 0.2353, 0.1091]])
tensor([[0.6595, 0.2326, 0.1079]])
tensor([[0.6538, 0.2

In [1]:
controller.stop()

NameError: name 'controller' is not defined

In [22]:
a = torch.tensor([[0.33, 0.58, 0.11], [0.1, 0.9, 0]])

dist = torch.distributions.Categorical(probs=a)
dist.entropy(), torch.sum(-a * torch.log(a + 1e-8), dim=1)

(tensor([0.9263, 0.3251]), tensor([0.9246, 0.3251]))